In [153]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [154]:
# Set the chromedriver path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [C:\Users\jl_ho\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


Visit the NASA website

In [155]:
# Visit the following URL
url = "https://mars.nasa.gov/news/"
browser.visit(url)


In [156]:
#Convert to soup object
html = browser.html
soup = bs(html, "html.parser")

In [157]:
news_title = soup.find('div', class_="content_title")
news_title = news_title.text.strip()
news_title

'Mars Now'

In [158]:
# find the first news title
    
news_paragraphs = soup.find('div', class_="article_teaser_body").get_text()
news_paragraphs

'Imagery has come down from Mars capturing a recent flight in which the rotorcraft flew farther and faster than ever before. '

Featured images

In [159]:
# Visit the following URL
url_2 = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(url_2)

In [160]:

html = browser.html
image_soup = bs(html, "html.parser")

In [161]:
featured_image = image_soup.find('img',class_= 'thumbimg').get('src')
featured_image

'image/mars/Icaria Fossae7.jpg'

In [162]:
image_url=f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html/{featured_image}'
image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html/image/mars/Icaria Fossae7.jpg'

Mars facts

In [163]:
df = pd.read_html('https://space-facts.com/mars/')[0]
df.head

<bound method NDFrame.head of                       0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers>

In [164]:
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [165]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

Hemispheres

In [166]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)

In [167]:
# List for the images and titles
hemisphere_images = []

#Get the links
links = browser.find_by_css('a.product-item img')

for h in range(len(links)):
    hemisphere = {}

    browser.find_by_css('a.product-item img')[h].click()
    
    sample = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample['href']

    hemisphere['title'] = browser.find_by_css('h2.title').text

    hemisphere_images.append(hemisphere)

    browser.back()

In [168]:
hemisphere_images

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]